# Shapes

## Basic Array

In [2]:
class Arr:
    def __init__(self, data):
        data = [item for item in data]
        self.dtype = dtype(data)
        self.data = [self.dtype(item) for item in data]

    def __str__(self):
        return (f"Arr: {self.data}")

    __repr__ = __str__

    def __len__(self):
        return len(self.data)

    def __iter__(self):
        return iter(self.data)

    ### Math

    def __add__(self, other):
        return zip_apply(self, other, self.dtype.__add__)

    def __sub__(self, other):
        return zip_apply(self, other, self.dtype.__sub__)

    def __mul__(self, other):
        return zip_apply(self, other, self.dtype.__mul__)

    def __truediv__(self, other):
        return zip_apply(self, other, self.dtype.__truediv__)

    def __abs__(self):
        return Arr(map(self.dtype.__abs__, self.data))


def zip_apply(left, right, f):
    # Length is the same
    assert len(left) == len(right), f'arrays are not of same shape'
    # Type is the same
    assert dtype(left) == dtype(right), f'Arrays are not of same dtype'
    # We can do the work
    result = [f(l, r) for (l, r) in zip(left, right)]
    return Arr(result)


def dtype(obj):
    """Returns the dtype of the array"""
    dtype = int
    for item in obj:
        itype = type(item)
        if itype == str:  # str is the largest, so dtype is str
            return str
        if itype == float:  # We haven't seen str by now so type is either float or int
            dtype = float
    return dtype

# Shape Method
Behind the scenes, the array is not "aware" of it having a shape, or that it has dimensions. Shape and dimensions are metadata which we use when we do operations.

In [4]:
class Arr(Arr):
    def __init__(self, data):
        data = [item for item in data]
        self.dtype = dtype(data)
        self.data = [self.dtype(item) for item in data]
        ### New Code
        self.size = len(self.data) # Because len will be something else
        self.shape = (self.size,)
        self.ndim = len(self.shape)

In [8]:
arr = Arr(range(16))
print(arr.shape)
print(arr.ndim)
print(arr)

(16,)
1
Arr: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [9]:
arr.shape = (4,4)
arr.ndim = 2
print (arr)

Arr: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


Not what we expected

## Better print for arrays

In [16]:
def print_array(arr):
    ret = "Arr: \n" # We will add more strings to this "main" string
    if arr.ndim < 2: 
        ret += str(arr.data)
    else:
        rows, cols = arr.shape
        for i, val in enumerate(arr.data):
            ret += str(val).center(6) # str.center - check it out.
            if i % cols == cols -1: # in "real" math this means i mod cols = -1
                ret += '\n'
    
    return ret

In [21]:
arr = Arr(range(16))
print(print_array(arr))

Arr: 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [22]:
arr.shape = (4,4)
arr.ndim = 2
print(print_array(arr))

Arr: 
  0     1     2     3   
  4     5     6     7   
  8     9     10    11  
  12    13    14    15  



**IT WORKS** Let's make this official.

In [23]:
Arr.__str__ = print_array
Arr.__repr__ = print_array

In [24]:
arr

Arr: 
  0     1     2     3   
  4     5     6     7   
  8     9     10    11  
  12    13    14    15  

# Reshape
Setting the `shape` is dangerous. It requires also changing the `ndim` and does check if the new shape makes sense

## (Bad) Examples

In [26]:
arr = Arr(range(16))
arr.shape = (4,4)
arr

Arr: 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [29]:
arr = Arr(range(16))
arr.ndim = 2
try:
    print (arr)
except Exception as e:
    print(e)

not enough values to unpack (expected 2, got 1)


In [33]:
arr = Arr(range(16))
arr.ndim = 2
arr.shape = (-3,4)
arr

Arr: 
  0     1     2     3   
  4     5     6     7   
  8     9     10    11  
  12    13    14    15  

## `Arr.reshape`
Requirements:  
* Change shape and ndim always together.
* Make sure rows * cols = size
* Have the option to change to either 1 dim or 2.
    * example: `arr.reshape(4,4)` or `arr.reshape(16)` both acceptable
* BONUS: have an option set one of the dimensions as -1 and have the array do the math.

In [34]:
# Ex
class Arr(Arr):
    def reshape(self, rows, cols=None):
        ret = Arr(self.data)
        # BOE
        if cols is None:
            ret = ret.reshape(rows, 1)
            ret.shape = (ret.shape[0],)
            ret.ndim = len(ret.shape)
            return ret
        elif cols == -1:
            assert self.size % rows == 0, f"rows must be divide {self.size} without a remainder"
            cols = self.size // rows
        elif rows == -1:
            assert self.size % cols == 0, f"cols must be divide {self.size} without a remainder"
            rows = self.size // cols
        assert rows * cols == len(self.data), (f"cannot reshape data with {self.size}"
                                               f" values to shape ({rows},{cols})")
        ret.shape = (rows, cols)
        ret.ndim = len(ret.shape)
        # EOE
        return ret
        

In [47]:
arr = Arr(range(16))
arr.reshape(-1)

Arr: 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [43]:
arr.reshape(4,4)

Arr: 
  0     1     2     3   
  4     5     6     7   
  8     9     10    11  
  12    13    14    15  

In [45]:
arr.reshape(2,-1)

Arr: 
  0     1     2     3     4     5     6     7   
  8     9     10    11    12    13    14    15  

In [46]:
arr.reshape(-1,2)

Arr: 
  0     1   
  2     3   
  4     5   
  6     7   
  8     9   
  10    11  
  12    13  
  14    15  

In [48]:
arr.reshape(5,3)

AssertionError: cannot reshape data with 16 values to shape (5,3)

In [49]:
arr.reshape(-1,7)

AssertionError: cols must be divide 16 without a remainder

## Protecting the shape with `__setitem__`
`__setitem__` runs before s propery is set to the instance

In [ ]:
# protected = ['shape', 'ndim']
# class Arr(Arr):
#     def __setitem__(self, item):
#         if item == 'shape'